In [1]:
import re
import numpy as np
import pandas as pd
import sqlite3

# Data Exploration

### Load dataset `Oscar Nominees and Winners`.

In [2]:
df = pd.read_csv('academy_awards.csv', encoding='ISO-8859-1')

In [3]:
df.head()

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010 (83rd),Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010 (83rd),Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN
2,2010 (83rd),Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN
3,2010 (83rd),Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,NaN,NaN,NaN,NaN,NaN,NaN
4,2010 (83rd),Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,NaN,NaN,NaN,NaN,NaN,NaN


### There are 6 unnamed columns at the end. We want to check if any of them have valid values that we need.

In [4]:
df['Unnamed: 5'].value_counts()

*                                                                                                               7
 error-prone measurements on sets. [Digital Imaging Technology]"                                                1
 discoverer of stars                                                                                            1
 D.B. "Don" Keele and Mark E. Engebretson has resulted in the over 20-year dominance of constant-directivity    1
 resilience                                                                                                     1
Name: Unnamed: 5, dtype: int64

In [5]:
df['Unnamed: 6'].value_counts()

*                                                                   9
 direct radiator bass style cinema loudspeaker systems. [Sound]"    1
 sympathetic                                                        1
 flexibility and water resistance                                   1
Name: Unnamed: 6, dtype: int64

In [6]:
df['Unnamed: 7'].value_counts()

 while requiring no dangerous solvents. [Systems]"    1
 kindly                                               1
*                                                     1
Name: Unnamed: 7, dtype: int64

In [7]:
df['Unnamed: 8'].value_counts()

 understanding comedy genius - Mack Sennett.""    1
*                                                 1
Name: Unnamed: 8, dtype: int64

In [8]:
df['Unnamed: 9'].value_counts()

*    1
Name: Unnamed: 9, dtype: int64

In [9]:
df['Unnamed: 10'].value_counts()

*    1
Name: Unnamed: 10, dtype: int64

### `Additional Info` column contains a few different formatting styles. We are going to clean this column.

In [10]:
df['Additional Info'].value_counts()

Metro-Goldwyn-Mayer                                                                                                   60
Walt Disney, Producer                                                                                                 57
Warner Bros.                                                                                                          42
John Williams                                                                                                         37
France                                                                                                                35
Alfred Newman                                                                                                         34
Italy                                                                                                                 26
Paramount                                                                                                             24
Gordon Hollingshead, Producer   

# Data Cleaning

### Deleting useless columns.

In [11]:
del_columns = ['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', ]
df = df.drop(del_columns, axis=1)

### Some String have pattern like {'name'}, we need to extract `name`.

In [12]:
def get_movie_name(row):
    if pd.notnull(row):
        return str(row).split('{\'')[0]

In [13]:
def get_character(row):
    if pd.notnull(row):
        res = str(row).split('{\'')
        if len(res) == 2:
            return res[1].rstrip('}\'')

In [14]:
df['Movie'] = df['Additional Info'].apply(get_movie_name)

In [15]:
df['Character'] = df['Additional Info'].apply(get_character)

In [16]:
df.head()

,Year,Category,Nominee,Additional Info,Won?,Movie,Character
0,2010 (83rd),Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,Biutiful,Uxbal
1,2010 (83rd),Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,True Grit,Rooster Cogburn
2,2010 (83rd),Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,The Social Network,Mark Zuckerberg
3,2010 (83rd),Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,The King's Speech,King George VI
4,2010 (83rd),Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,127 Hours,Aron Ralston


## What's the best way to handle awards ceremonies that included movies from 2 years?

### E.g. see 1927/28 (1st) in the Year column.

In [17]:
df.iloc[10136]

Year                                                1927/28 (1st)
Category           Unique and Artistic Picture (archaic category)
Nominee                                    Paramount Famous Lasky
Additional Info                                             Chang
Won?                                                           NO
Movie                                                       Chang
Character                                                    None
Name: 10136, dtype: object

### Appending these data to the end. recording the previous row number.

In [18]:
# last 10136
total_row = df.shape[0]
print(total_row)
df = df.append( df[ (df['Year'].str.len() > 4) & (df['Year'].str.contains('/')) ], ignore_index=True)

10137


### Dealing with new data.

In [19]:
for i in range(total_row, df.shape[0]):
    search_year = re.search('(\d{2})\d{2}/(\d{2})', df.iloc[i]['Year'])
    if search_year:
        df.iloc[i]['Year'] = search_year.group(1) + search_year.group(2)

In [20]:
df[total_row:].head()

,Year,Category,Nominee,Additional Info,Won?,Movie,Character
10137,1933,Actor -- Leading Role,Leslie Howard,Berkeley Square {'Peter Standish'} [came in 3rd],NO,Berkeley Square,Peter Standish'} [came in 3rd]
10138,1933,Actor -- Leading Role,Charles Laughton,The Private Life of Henry VIII {'Henry VIII'},YES,The Private Life of Henry VIII,Henry VIII
10139,1933,Actor -- Leading Role,Paul Muni,I Am a Fugitive from a Chain Gang {'James Alle...,NO,I Am a Fugitive from a Chain Gang,James Allen'} [came in 2nd]
10140,1933,Actress -- Leading Role,Katharine Hepburn,Morning Glory {'Eva Lovelace'},YES,Morning Glory,Eva Lovelace
10141,1933,Actress -- Leading Role,May Robson,Lady for a Day {'Apple Annie'} [came in 2nd],NO,Lady for a Day,Apple Annie'} [came in 2nd]


### Dealing with old data. Just get the first four digits.

In [21]:
df["Year"] = df["Year"].str[:4]

### Convert the `Year` column to `int` data type.

In [22]:
df['Year'] = df['Year'].astype(int)

In [23]:
df.head()

,Year,Category,Nominee,Additional Info,Won?,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,127 Hours,Aron Ralston


### We only need the data after the year 2000.

In [24]:
later_than_2000 = df[df['Year'] > 2000]

In [25]:
later_than_2000.head()

,Year,Category,Nominee,Additional Info,Won?,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,127 Hours,Aron Ralston


### There are many different types of `Category`, we only interest in four awards.

In [26]:
later_than_2000['Category'].value_counts()

Writing                                                        100
Scientific and Technical (Technical Achievement Award)          70
Best Picture                                                    60
Foreign Language Film                                           50
Short Film (Live Action)                                        50
Music (Scoring)                                                 50
Actress -- Leading Role                                         50
Directing                                                       50
Cinematography                                                  50
Costume Design                                                  50
Short Film (Animated)                                           50
Actor -- Supporting Role                                        50
Actor -- Leading Role                                           50
Film Editing                                                    50
Sound                                                         

In [27]:
award_categories = ['Actor -- Leading Role', 'Actor -- Supporting Role', 'Actress -- Leading Role', 'Actress -- Supporting Role']

In [28]:
nominations = later_than_2000[later_than_2000['Category'].isin(award_categories)]

In [29]:
nominations.head()

,Year,Category,Nominee,Additional Info,Won?,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,127 Hours,Aron Ralston


### Cleaning up the `Won` column. Converting `YES` to `1`, `NO` to `0`.

In [30]:
nominations['Won?'] = nominations['Won?'].map({'YES': 1, 'NO': 0})

/Users/yuzezou/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [31]:
nominations.head()

,Year,Category,Nominee,Additional Info,Won?,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},0,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},0,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},0,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},1,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},0,127 Hours,Aron Ralston


### Sorting rows by `Year`.

In [32]:
nominations = nominations.sort_values(by='Year')

In [33]:
nominations.head()

,Year,Category,Nominee,Additional Info,Won?,Movie,Character
1176,2001,Actress -- Supporting Role,Kate Winslet,Iris {'Young Iris Murdoch'},0,Iris,Young Iris Murdoch
1157,2001,Actor -- Leading Role,Russell Crowe,A Beautiful Mind {'John Nash'},0,A Beautiful Mind,John Nash
1158,2001,Actor -- Leading Role,Sean Penn,I Am Sam {'Sam Dawson'},0,I Am Sam,Sam Dawson
1159,2001,Actor -- Leading Role,Will Smith,Ali {'Muhammad Ali'},0,Ali,Muhammad Ali
1160,2001,Actor -- Leading Role,Denzel Washington,Training Day {'Alonzo'},1,Training Day,Alonzo


### Renaming `Won?` to `Won`

In [34]:
nominations = nominations.rename(columns={'Won?': 'Won'})

In [35]:
final_nominations = nominations.drop('Additional Info', axis=1)

In [36]:
final_nominations = final_nominations.reset_index(drop=True)

In [37]:
final_nominations.head()

,Year,Category,Nominee,Won,Movie,Character
0,2001,Actress -- Supporting Role,Kate Winslet,0,Iris,Young Iris Murdoch
1,2001,Actor -- Leading Role,Russell Crowe,0,A Beautiful Mind,John Nash
2,2001,Actor -- Leading Role,Sean Penn,0,I Am Sam,Sam Dawson
3,2001,Actor -- Leading Role,Will Smith,0,Ali,Muhammad Ali
4,2001,Actor -- Leading Role,Denzel Washington,1,Training Day,Alonzo


In [38]:
final_nominations.dtypes

Year          int64
Category     object
Nominee      object
Won           int64
Movie        object
Character    object
dtype: object

# Exporting Pandas DataFrame to SQLite3 table.

### Deleting the file if the file name exist.

In [39]:
!rm -rf nominations.db

In [40]:
conn = sqlite3.connect('nominations.db')

final_nominations.to_sql(name='nominations', con=conn, index=False)

In [41]:
!ls nominations.db

nominations.db


### Printing `nominations` schema.

In [42]:
schema = conn.execute('pragma table_info(nominations)').fetchall()
schema

[(0, 'Year', 'INTEGER', 0, None, 0),
 (1, 'Category', 'TEXT', 0, None, 0),
 (2, 'Nominee', 'TEXT', 0, None, 0),
 (3, 'Won', 'INTEGER', 0, None, 0),
 (4, 'Movie', 'TEXT', 0, None, 0),
 (5, 'Character', 'TEXT', 0, None, 0)]

### Showing some data.

In [43]:
first_ten = conn.execute('select * from nominations limit 10;').fetchall()
first_ten = conn.execute('select * from nominations limit 10;').fetchall()
for s in schema:
    print('|  ', s[1], ' ',end='')
print('|')
for li in first_ten:
    print(li)

|   Year  |   Category  |   Nominee  |   Won  |   Movie  |   Character  |
(2001, 'Actress -- Supporting Role', 'Kate Winslet', 0, 'Iris ', 'Young Iris Murdoch')
(2001, 'Actor -- Leading Role', 'Russell Crowe', 0, 'A Beautiful Mind ', 'John Nash')
(2001, 'Actor -- Leading Role', 'Sean Penn', 0, 'I Am Sam ', 'Sam Dawson')
(2001, 'Actor -- Leading Role', 'Will Smith', 0, 'Ali ', 'Muhammad Ali')
(2001, 'Actor -- Leading Role', 'Denzel Washington', 1, 'Training Day ', 'Alonzo')
(2001, 'Actor -- Leading Role', 'Tom Wilkinson', 0, 'In the Bedroom ', 'Matt Fowler')
(2001, 'Actor -- Supporting Role', 'Jim Broadbent', 1, 'Iris ', 'John Bayley')
(2001, 'Actor -- Supporting Role', 'Ethan Hawke', 0, 'Training Day ', 'Jake')
(2001, 'Actor -- Supporting Role', 'Ben Kingsley', 0, 'Sexy Beast ', 'Don Logan')
(2001, 'Actor -- Supporting Role', 'Ian McKellen', 0, 'The Lord of the Rings: The Fellowship of the Ring ', 'Gandalf')


# Creating tables.

## Creating `ceremonies` table.

In [44]:
cere_schema = conn.execute('pragma table_info(ceremonies)').fetchall()
if len(cere_schema) == 0:
    sql = 'create table ceremonies( \
        id integer primary key, \
        Year integer, \
        Host text \
        )'
    conn.execute(sql)
    
    # Creating a list for each awards ceremony.
    years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]

    conn.executemany('insert into ceremonies (Year, Host) values(?, ?)', years_hosts)

### Validating results.

In [45]:
conn.execute('pragma table_info(ceremonies)').fetchall()

[(0, 'id', 'integer', 0, None, 1),
 (1, 'Year', 'integer', 0, None, 0),
 (2, 'Host', 'text', 0, None, 0)]

In [46]:
conn.execute('select * from ceremonies').fetchall()

[(1, 2010, 'Steve Martin'),
 (2, 2009, 'Hugh Jackman'),
 (3, 2008, 'Jon Stewart'),
 (4, 2007, 'Ellen DeGeneres'),
 (5, 2006, 'Jon Stewart'),
 (6, 2005, 'Chris Rock'),
 (7, 2004, 'Billy Crystal'),
 (8, 2003, 'Steve Martin'),
 (9, 2002, 'Whoopi Goldberg'),
 (10, 2001, 'Steve Martin'),
 (11, 2000, 'Billy Crystal')]

### Turn on foreign key constraints.

In [47]:
conn.execute('pragma foreign_keys = ON;')

## Changing `nominations` table.

### Creating `nominations_two` table.

In [48]:
nom_schema = conn.execute('pragma table_info(nominations_two)').fetchall()
if len(nom_schema) == 0:
    sql = 'create table nominations_two( \
            id integer primary key, \
            category text, \
            nominee text, \
            movie text, \
            character text, \
            won text, \
            ceremony_id integer, \
            foreign key(ceremony_id) references ceremonies(id));'
    conn.execute(sql)

    # insert values
    sql = 'insert into nominations_two(category, nominee, movie, character, won, ceremony_id) \
            SELECT nominations.category, nominations.nominee, nominations.movie, nominations.character, nominations.won, ceremonies.id \
            FROM nominations \
            INNER JOIN ceremonies ON \
            nominations.year == ceremonies.year;'
    conn.execute(sql)

### Printing results.

In [49]:
nominations_two = conn.execute('select * from nominations_two limit 5;').fetchall()
for l in nominations_two:
    print(l)

(1, 'Actress -- Supporting Role', 'Kate Winslet', 'Iris ', 'Young Iris Murdoch', '0', 10)
(2, 'Actor -- Leading Role', 'Russell Crowe', 'A Beautiful Mind ', 'John Nash', '0', 10)
(3, 'Actor -- Leading Role', 'Sean Penn', 'I Am Sam ', 'Sam Dawson', '0', 10)
(4, 'Actor -- Leading Role', 'Will Smith', 'Ali ', 'Muhammad Ali', '0', 10)
(5, 'Actor -- Leading Role', 'Denzel Washington', 'Training Day ', 'Alonzo', '1', 10)


### Deleting `nominations` table.

In [50]:
conn.execute('drop table nominations;')

### Renaming table name.

In [51]:
conn.execute('alter table nominations_two rename to nominations;')

## Creating `movies_actors` join tables

In [52]:
movies_actors_schema = conn.execute('pragma table_info(movies_actors);').fetchall()
if len(movies_actors_schema) == 0:
    # Creating `movies` table
    sql = 'create table movies( \
            id integer primary key, \
            movie text)'
    conn.execute(sql)
    
    # Creating `actors` table
    sql = 'create table actors( \
            id integer primary key, \
            actor text);'
    conn.execute(sql)
    
    # Creating `movies_actors` table
    sql = 'create table movies_actors( \
            id integer primary key, \
            movie_id integer references movies(id), \
            actor_id integer references actors(id) \
            );'
    conn.execute(sql)
    
    # Filling `movies`
    sql = 'insert into movies (movie) \
            select distinct movie from nominations;'
    conn.execute(sql)
    
    # Filling `actors`
    sql = 'insert into actors (actor) \
            select distinct character from nominations;'
    conn.execute(sql)
    
    # Filling `movies_actors`
    sql = 'insert into movies_actors (movie_id, actor_id) \
            select movies.id, actors.id \
            from movies join nominations \
            on movies.movie = nominations.movie \
            join actors \
            on nominations.character = actors.actor;'
    conn.execute(sql)

In [53]:
### Checking values.

In [54]:
sql = 'select * from movies limit 5;'
conn.execute(sql).fetchall()

[(1, 'Iris '),
 (2, 'A Beautiful Mind '),
 (3, 'I Am Sam '),
 (4, 'Ali '),
 (5, 'Training Day ')]

In [55]:
sql = 'select * from actors limit 5;'
conn.execute(sql).fetchall()

[(1, 'Young Iris Murdoch'),
 (2, 'John Nash'),
 (3, 'Sam Dawson'),
 (4, 'Muhammad Ali'),
 (5, 'Alonzo')]

In [56]:
sql = 'select * from movies_actors limit 5;'
conn.execute(sql).fetchall()

[(1, 1, 19), (2, 1, 7), (3, 1, 1), (4, 2, 15), (5, 2, 2)]

In [57]:
conn.close()